In [ ]:
# Automating Existing Processes using Python
# May 2021
#
# Miklos Nadas, GISP
# 
# City of Cleveland

# Agenda
#
# 1. Introduction
#    a. Python & Jupyter
#    b. Miklos Nadas, GISP
#    c. City of Cleveland
#
# 2. My Automation Steps
#    a. Study the data
#    b. Manually Perform Process
#    c. Investigate Individual Steps (testing with API)
#    d. Decision to Automate
#        i.   Can it be automated?
#        ii.  Will the process be repeated?
#        iii. How much time will it take to automate?
#    e. Automate!    
#
# 3. Photo Management
#    a. Manual Process
#    b. Investigate - PIL library
#    c. Issues - Task Scheduler
# 
# 4. Recreating address locators
#    a. Manual Process
#    b. Investigate - arcpy library (ArcGIS API)
#    c. Issues - Cannot automate all properties
#
# 5. Cityworks and Survey123 Integration
#    a. Manual Process
#    b. Investigate - cwpy library (Cityworks API)
#    c. Issues - Not end user friendly
# 
# 6. Updating an ArcGIS Online Operations Dashboard
#    a. Manual Process
#    b. Investigate - arcgis library (ArcGIS Online API)
#    c. Issues - Complicated
#
# 7. Closing Thoughts and Moving forward

In [ ]:
# Python - open source programming language - utilized by Esri
# Jupyter Notebooks - interactive data science and scientific computing

In [ ]:
# The libraries used for this presentation

from arcgis import GIS                       # Mapping, spatial analysis, data science, geospatial AI and automation
from IPython.display import Image as jImage  # Displays images in Jupyter
import webbrowser                            # Web-browser controller
import pandas                                # Data analysis and manipulation tool
import os                                    # Miscellaneous operating system interfaces
import shutil                                # High-level operations on files, such as copying and removal
from PIL import Image                        # Imaging library
# import arcpy                               # ArcGIS API Library

In [ ]:
# My work history

url = r'https://www.linkedin.com/in/miklosnadas'
webbrowser.open(url)

In [ ]:
# City of Cleveland logo

city_image = r'C:\data\NEOGIS_2021\pics\City-Logo.jpg'
jImage(filename=city_image, width=200)

In [ ]:
# Connecting to ArcGIS Online

gis = GIS(r'http://www.arcgis.com/', 'user', 'password')

In [ ]:
# Creating a Map to show the location of Cleveland, OH

map_cleveland = gis.map('Ohio')
# Cleveland Point
cleve_pnt = '3e9ce2e14bb3452eb9ce3d6eab9b85a4'

cleve_pnt_fs = gis.content.get(cleve_pnt)
map_cleveland.add_layer(cleve_pnt_fs)

In [ ]:
map_cleveland

In [ ]:
# Adding the Cleveland Boundary

cleve_boundary = 'bd1f0a17a11d4610869ae3abbeb3646a'
cleve_boundary_fs = gis.content.get(cleve_boundary)

map_cleveland.add_layer(cleve_boundary_fs.layers[0])
map_cleveland.extent = [[-81.6, 41.7], [-81.8, 41.3]]

In [ ]:
# Population of Cleveland using USA Block Groups
block_groups = 'b963313229bd44d5825a8f7352b09ce4'

block_fs = gis.content.get(block_groups)

sdf = pandas.DataFrame.spatial.from_layer(block_fs.layers[0])

In [ ]:
sdf.head()

In [ ]:
# Population of Cleveland

print("2012 Population Cleveland from Census Block Groups: {}".format('{:,}'.format(sdf.loc[sdf['WITHINCOC'] == 'Yes', 'POP2012'].sum())))

In [ ]:
# 2. My Automation Steps
#    a. Study the data
#    b. Manually Perform Process
#    c. Investigate Individual Steps (testing with API)
#    d. Decision to Automate
#        i.   Can it be automated?
#        ii.  Will the process be repeated?
#        iii. How much time will it take to automate?
#    e. Automate!   

In [ ]:
# 3. Photo Management
#    a. Manual Process - IrfanView
#    b. Investigate - PIL library
#    c. Issues - Task Scheduler

In [ ]:
# Photo to shrink

photo_loc = r'C:\data\NEOGIS_2021\pics\hydrant.jpg'
jImage(filename=photo_loc, width=400)

In [ ]:
# Check size of photo

bytesize = os.stat(photo_loc).st_size
print("Byte size of photo: {}".format('{:,}'.format(bytesize)))
print("MB size of photo: {}".format('{:,}'.format(bytesize/1000000)))

In [ ]:
# If size of photo is over 1 MB, shrink it
# If less than 1 MB, skip it

if bytesize > 1000000:
    print("Too Large! Shrink Photo!")
else:
    print("Skip photo")

In [ ]:
# Define the back up location of the photo

backup_photo = photo_loc[:-4] + "_backup" + photo_loc[-4:]
print(backup_photo)

In [ ]:
# Copy the original photo to the backup location

shutil.copyfile(photo_loc, backup_photo)

In [ ]:
# Display the backup photo

jImage(filename=backup_photo, width=400)

In [ ]:
# PIL API to access photo information

img = Image.open(photo_loc)

In [ ]:
# Dimensions of the photo

img.size

In [ ]:
# Size of one side of photo should be 1280
# Adjust other side of photo appropriately

img_size = img.size
if img_size[0] > img_size[1]:
    factor = 1280 / img_size[0]
else:
    factor = 1280 / img_size[1]
print(factor)

In [ ]:
# New dimensions of photo

print("Size of side 0: {}".format(img_size[0]))
print("Size of side 1: {}".format(img_size[1]))
print("Reduced size of side 0: {}".format(img_size[0] * factor))
print("Reduced size of side 0: {}".format(img_size[1] * factor))

In [ ]:
# Create a new shrunken image in virtual memory

new_img = img.resize((int(img_size[0] * factor), int(img_size[1] * factor)), Image.ANTIALIAS)

In [ ]:
# Add metadata to new image
# Overwrite original image

exifdump = None
try:
    exifdump = img.info['exif']
    new_img.save(photo_loc, img.format, optimize=True, exif=exifdump)
    print("Shrank with exif: {}".format(photo_loc))
except:
    print("No Exif Info: {}".format(photo_loc))
    new_img.save(photo_loc, img.format, optimize=True)
    print("Shrank: {}".format(photo_loc))

In [ ]:
# Check size of new image and compare to original image

bytesize = os.stat(photo_loc).st_size
print("Byte size of shrunk photo: {}".format('{:,}'.format(bytesize)))
print("MB size of shrunk photo: {}".format('{:,}'.format(bytesize/1000000)))
bytesize_backup = os.stat(backup_photo).st_size
print("Byte size of original photo: {}".format('{:,}'.format(bytesize_backup)))
print("MB size of original photo: {}".format('{:,}'.format(bytesize_backup/1000000)))

In [ ]:
# Review image to see if it is still viewable

photo_loc = r'C:\data\NEOGIS_2021\pics\hydrant.jpg'
jImage(filename=photo_loc, width=400)

In [ ]:
# 3. Photo Management
#    a. Combine pieces
#        i.   Check size of drive and email (full, empty, total)
#        ii.  Shrink the photos with detailed logging, checking, backups
#        iii. Delete 2 week old backups with detailed logging and verifying
#        iv.  Notify users of too many photos or large attachments
#        v.   Check size of drive and email (full, empty, total)
#    b. Issues - Task Scheduler

In [ ]:
# 4. Recreating address locators
#    a. Manual Process
#    b. Investigate - arcpy library (ArcGIS API)
#    c. Issues - Cannot automate all properties

In [ ]:
# Run the Create Address Locator geoprocessing tool in ArcMap

addr1_photo = r'C:\data\NEOGIS_2021\pics\Address_1.png'
jImage(filename=addr1_photo, width=400)

In [ ]:
# Copy as Python Snippet

addr2_photo = r'C:\data\NEOGIS_2021\pics\Address_2.png'
jImage(filename=addr2_photo, width=400)

In [ ]:
# Address Point feature class

f = open(r"C:\data\NEOGIS_2021\sdeloc.txt", "r")
sdeaddr = f.read()

address_points = sdeaddr

In [ ]:
# Export location of address locator

outfldr = r'C:\data\NEOGIS_2021\addr'
loc_addr_pnts_name = 'Loc_Addr_Pnts'
addr_path = os.path.join(outfldr, loc_addr_pnts_name)

In [ ]:
# Python Snippet from Create Address Locator tool

arcpy.CreateAddressLocator_geocoding(in_address_locator_style="US Address - Single House",
                                     in_reference_data=address_points + " 'Primary Table'",
                                     in_field_map="'Point Address ID' ADDRESSID VISIBLE NONE;'Street ID' <None> VISIBLE NONE;'*House Number' ADDR_NUM VISIBLE NONE;Side <None> VISIBLE NONE;'Full Street Name' CAT_NAME VISIBLE NONE;'Prefix Direction' PRE_DIR VISIBLE NONE;'Prefix Type' <None> VISIBLE NONE;'*Street Name' STR_NAME VISIBLE NONE;'Suffix Type' STR_TYPE VISIBLE NONE;'Suffix Direction' SUF_DIR VISIBLE NONE;'City or Place' CITY VISIBLE NONE;County COUNTY VISIBLE NONE;State STATE VISIBLE NONE;'State Abbreviation' STATE VISIBLE NONE;'ZIP Code' ZIP VISIBLE NONE;'Country Code' <None> VISIBLE NONE;'3-Digit Language Code' <None> VISIBLE NONE;'2-Digit Language Code' <None> VISIBLE NONE;'Admin Language Code' <None> VISIBLE NONE;'Block ID' <None> VISIBLE NONE;'Street Rank' <None> VISIBLE NONE;'Display X' <None> VISIBLE NONE;'Display Y' <None> VISIBLE NONE;'Min X value for extent' <None> VISIBLE NONE;'Max X value for extent' <None> VISIBLE NONE;'Min Y value for extent' <None> VISIBLE NONE;'Max Y value for extent' <None> VISIBLE NONE;'Additional Field' <None> VISIBLE NONE;'Altname JoinID' <None> VISIBLE NONE;'City Altname JoinID' <None> VISIBLE NONE",
                                     out_address_locator=addr_path,
                                     config_keyword="",
                                     enable_suggestions="ENABLED")

In [ ]:
# Additional properties not accessible by the API

addr3_photo = r'C:\data\NEOGIS_2021\pics\Address_3.png'
jImage(filename=addr3_photo, width=400)

In [ ]:
# Original XML copied into new Address Locator

addr4_photo = r'C:\data\NEOGIS_2021\pics\Address_4.png'
jImage(filename=addr4_photo, width=700)

In [ ]:
# Copy those properties from a previously configured address locator's XML

defaultxml_path = r'C:\data\NEOGIS_2021\addr\default\Loc_Addr_Pnts_default.loc.xml'

if os.path.exists(addr_path + '.loc.xml'): os.remove(addr_path + '.loc.xml')
shutil.copy2(defaultxml_path, outfldr)
os.rename(os.path.join(outfldr, 'Loc_Addr_Pnts_default.loc.xml'), os.path.join(outfldr, loc_addr_pnts_name + '.loc.xml'))

In [ ]:
# 4. Recreating address locators
#    c. Issues - Cannot copy XML for composite address locators

In [ ]:
# 5. Cityworks and Survey123 Integration
#    a. Manual Process
#    b. Investigate - cwpy library (Cityworks API)

In [ ]:
# All Cityworks instances install the API documentation

url = r'https://www.cityworks.com/cityworks/apidocs/'
webbrowser.open(url)

In [ ]:
# Cityworks libraries used to create API calls

import cwpy.cwServices, cwpy.cwMessagesAMS # to get the Cityworks token
import requests, json # to make the rest of the Cityworks API Calls

In [ ]:
# Function to convert Python dictionary to JSON.

def data_to_json(data_dict): 
    token = cw_token
    json_data = json.dumps(data_dict, separators=(",",":"))
    if len(list(token)) == 0:
        params = {"data": json_data}
    else:
        params = {"token": token, "data": json_data}
    return params

In [ ]:
# Function to make an API call.

def make_request(url, params):  
    response = requests.get(url, params=params)
    return json.loads(response.text)

In [ ]:
# City of Cleveland Sandbox Instance

base_url = "https://www.cityworks.com/cityworks"

In [ ]:
# To get the Cityworks Token

services = cwpy.cwServices.Services()
services.url = base_url
base_url = base_url + r'/Services/'
username = "user"
password ="password"
auth_response = services.authenticate(username, password)
cw_token = auth_response["Value"]["Token"]

In [ ]:
# CW Part 1. Create CW Inspection

insp_template = 70 # Created an DOS Asset Photo Inspection Template
entity_type = 'GUARDRAILS'

url = base_url + 'Ams/Inspection/Create'
data = {"EntityType": entity_type, "InspTemplateId": insp_template}
parameters = data_to_json(data)
insp_create = make_request(url, parameters)

insp_id = insp_create['Value']['InspectionId']

# print(insp_create['Value'])

In [ ]:
print("InspectionID: {}".format(insp_create['Value']['InspectionId']))
inspection_url = r'https://cityworks-sb.clevelandgis.net/cityworksSB155miklos/Workmanagement/InspectionEdit.aspx?InspectionId=' + str(insp_create['Value']['InspectionId'])
webbrowser.open(inspection_url)

In [ ]:
# CW Part 2. Update CW inspection

url = base_url + 'Ams/Inspection/Update'

data = {"InspectionId": insp_id,
        "DateSubmitTo": '0001-01-01 00:00:00',
        "InspectionDate": '2021-04-16 09:00:00', # today's date
        "InspectedBy": str(12698), # inspector UID
        }

parameters = data_to_json(data)
insp_update = make_request(url, parameters)
print('Status: {}'.format(str(insp_update['Status'])))

In [ ]:
# CW Part 3. Add entity to connect to GIS

url = base_url + 'Ams/Inspection/AddEntity'

data = {
    "EntityType": entity_type,
    "InspectionId": insp_id,
    "EntityUid": '53254'
    }
parameters = data_to_json(data)
entity_add = make_request(url, parameters)

print('Status: {}'.format(str(entity_add['Status'])))

In [ ]:
# CW Part 4. Attach a list of photos

filepath = r'C:\data\NEOGIS_2021\pics'
photo_list = [os.path.join(filepath, 'guardrail_photo.jpg'), os.path.join(filepath, 'jimi-hendrix.jpg')]

for filepath in photo_list:
    attach = open(filepath, "rb")
    attaches = {"file": (os.path.basename(filepath), attach)}
    data = {
        "InspectionId": insp_id,
    }

    url = base_url + 'Ams/Attachments/AddInspectionAttachment'
    parameters = data_to_json(data)
    response = requests.post(url=url, files=attaches, data=parameters)
    attach_response = json.loads(response.text)
    print('Status: {}'.format(str(attach_response['Status'])))

In [ ]:
# CW Part 5. Close Inspection

url = base_url + 'Ams/Inspection/Close'
data = {"InspectionIds": [insp_id]}
parameters = data_to_json(data)
insp_close = make_request(url, parameters)
print('Status: {}'.format(str(insp_close['Status'])))

In [ ]:
# 5. Cityworks and Survey123 Integration
#    c. Issues
#       i.   Did not provide robust error checking
#       ii.  Only usable by a Pythonista

In [ ]:
# 6. Updating an ArcGIS Online Operations Dashboard
#    a. Manual Process - due to security
#    b. Investigate - ArcGIS Online API
#    c. Issues - Complicated

In [ ]:
# Coronavirus Dashboard - ArcGIS Online Operations Dashboard
#
# 1. Provide 2 spreadsheets of confirmed and probable cases
# 2. Schema changes happened by accident and without notification
# 3. Python Steps
#    a. Merge Probable and Confirmed cases
#       i.   Update table headers (column names) to standard names
#       ii.  Merge using Pandas
#    b. Aggregate cases to ward and zip codes
#       i.   Geocode cases
#       ii.  Calculate latitude/longitude
#       iii. Merge households
#       iv.  Spatially joined with wards
#       v.   Spatially joined with zip codes
#       vi.  Export jpg maps
#    c. Added and Updated cases to ArcGIS Online
#       i.   Use pandas and ArcGIS Online API
#       ii.  Download current ArcGIS Online of Cases Table
#       iii. Compare downloaded table with latest spreadsheet
#       iv.  Add if case is new
#       v.   Update if case is different
#    d. Update Internal Dashboard
#       i.   Business Intelligence (BI) want to drill down in data
#       ii.  Add Ohio and Cuyahoga confirmed and probable numbers
#       iii. Update cases per zip code
#    e. Update Public Dashboard
#       i.   Static, no BI integration, no data drill down
#       ii.  Calculate statistics (i.e. average age or % male vs female)
#       iii. Update numbers
# 4. Issues
#    a. Could not update map, needed to manually trigger symbology update
#    b. Manual and Automated process was difficult

In [ ]:
# Cleveland Department of Public Health - Covid Dashboard

url = r'https://www.arcgis.com/apps/dashboards/03b3bfc3eb82483b8c2c072e2abaefd2'
webbrowser.open(url)

In [ ]:
# Final Thoughts on Automating Existing Processes using Python
#
# 1. Automation Steps
#    a. Manual Perform the Process
#    b. Investigate
#    c. Decision
#    d. Automate

In [ ]:
# Moving forward at Cleveland
#
# 1. Upgrade Cityworks
# 2. Automate quality assurance testing for future upgrades and enhancements (more Python!)
# 3. Postman for additional assistance and increasing API efficiency

In [ ]:
# github link with Jupyter Notebook

github.com/milkor56/conferences

In [ ]:
# Contact Information
#
# Miklos Nadas, GISP
# City of Cleveland
# miklos_nadas@clevelandwater.com
#